# Лабораторная работа №4
Деревья решений
Выполнила Фонград Анастасия КИ21-16/1б

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, Lars, TweedieRegressor, BayesianRidge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings("ignore")

## Предварительная обработка исходных данных